### Held Karp Implementation

In [39]:
import math
from functools import cache

def held_karp_solver(adj_matrix):
    """
    Solve the (modified) traveling salesman problem using Held-Karp algorithm (Dynamic Programming).
    
    Parameters:
    adj_matrix: adjacency matrix where adj_matrix[i][j] is the distance from node i to node j
    
    Returns:
    path: list representing the shortest path from the first node to the last node
    distance: the shortest distance for the path
    """
    
    n = len(adj_matrix)
    if n == 0:
        return [], 0
    
    @cache
    def g(S, k):
        """
        Recursive function to find the minimum distance to reach node k from node 0 after visiting all nodes in set S.
        
        Parameters:
        S: bit-mask representing the set of nodes to visit in-between (ex. node 1, 3 = 0b101)
        k: the last node to visit
        
        Returns:
        dist: the minimum distance to reach node k from node 0 through all nodes in S
        prev: the previous node before reaching k
        """
        if S == 0:
            return adj_matrix[0][k], 0
        
        # Evaluate minimum distance by checking all possible previous nodes in the set S
        return min(
            (g(S ^ (1 << (i - 1)), i)[0] + adj_matrix[i][k], i) # dist(S - {i}, i) + dist(i, k)
            for i in range(1, n) if S & (1 << (i - 1)) # For all i in S
        )

    # Initialize S as all nodes except node 0 and node n-1
    S = (1 << (n - 1)) - 1 
    dist, prev = g(S, n - 1)

    # Reconstruct the path by backtracking from the last node
    path = []
    while S:
        path.append(prev)
        S ^= 1 << (prev - 1) # S = S - {prev}
        _, prev = g(S, prev)

    return [0] + path[::-1], dist


Implementation using python set for performance comparison

In [40]:
import math
from functools import cache

# Parameters:
#   adj_matrix: adjacency matrix of the graph where adj_matrix[i][j] represents the distance from node i to node j
# Return:
#   path: the shortest path from the first node to the last node
#   distance: the distance of the shortest path
def held_karp_solver_using_set(adj_matrix):
    # Find the shortest path
    n = len(adj_matrix)

    if n == 0:
        return [], 0

    # Dynamic programming function
    # Parameter:
    #   S: frozenset of nodes to visit
    #   k: the last node to visit
    # Return:
    #   dist: the distance from the first node to k while visiting all nodes in S exactly once
    #   prev: the previous node to visit before k
    @cache
    def g(S, k):
        if not S:  # If there are no nodes to visit before k
            return adj_matrix[0][k], 0  # dist(0, k) = adj_matrix[0][k], prev(0, k) = 0
        
        # Find the minimum distance from the first node to k while visiting all nodes in S exactly once
        candidates = [
            (g(S - frozenset([i]), i)[0] + adj_matrix[i][k], i)  # dist(S - {i}, i) + dist(i, k)
            for i in S
        ]
        return min(candidates, key=lambda x: x[0])
    
    S = frozenset(range(1, n))  # All nodes except the first and the last
    dist, prev = g(S, n - 1)  # Start from all nodes and end at the last node
    path = []
    while S:
        path.append(prev)
        S = S - frozenset([prev])  # Remove the last visited node from the set
        _, prev = g(S, prev)
    path.reverse()
    path = [0] + path  # Include the starting node
    
    return path, dist


### BF Method

In [41]:
from itertools import permutations

# A BF solver
def bf_solver(adj_matrix):
    if len(adj_matrix) == 0:
        return [], 0
    elif len(adj_matrix) == 1:
        return [0], 0
    
    min_dist = math.inf
    min_path = None
    for perm in permutations(range(1, len(adj_matrix)-1)): # [1, 2, ..., n-2]
        path = [0] + list(perm) + [len(adj_matrix)-1]
        dist = sum(adj_matrix[path[i]][path[i+1]] for i in range(len(path)-1))
        if dist < min_dist:
            min_dist = dist
            min_path = path
    return min_path, min_dist

### ILP Method (Not Stable)

In [42]:
%pip install pulp

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
import pulp

# WARNING: This implementation is not stable.
def tsp_ilp_solver(adj_matrix):
    """
    Solve the traveling salesman problem using ILP (Integer Linear Programming)
    to find the shortest path from the first node to the last node.
    
    Parameters:
    adj_matrix: adjacency matrix where adj_matrix[i][j] is the distance from node i to node j
    
    Returns:
    path: list representing the shortest path from the first node to the last node
    distance: the shortest distance for the path
    """
    
    n = len(adj_matrix)
    if n == 0:
        return [], 0

    # Create the ILP problem
    prob = pulp.LpProblem("TSP", pulp.LpMinimize)

    # Decision variables: x[i,j] = 1 if edge (i,j) is in the path, otherwise 0
    x = pulp.LpVariable.dicts("x", [(i, j) for i in range(n) for j in range(n) if i != j], cat='Binary')

    # Objective: minimize the sum of the distances for the selected edges
    prob += pulp.lpSum(adj_matrix[i][j] * x[i, j] for i in range(n) for j in range(n) if i != j)

    # Constraints:
    # 1. Node 0 (start node) has exactly one outgoing edge
    prob += pulp.lpSum(x[0, j] for j in range(1, n)) == 1

    # 2. Node n-1 (end node) has exactly one incoming edge
    prob += pulp.lpSum(x[i, n-1] for i in range(n-1)) == 1

    # 3. All other nodes (1 to n-2) must have one incoming and one outgoing edge
    for i in range(1, n-1):
        prob += pulp.lpSum(x[i, j] for j in range(n) if i != j) == 1  # Outgoing edges
        prob += pulp.lpSum(x[j, i] for j in range(n) if i != j) == 1  # Incoming edges

    # 4. Eliminate subtours using Miller-Tucker-Zemlin (MTZ) formulation
    u = pulp.LpVariable.dicts("u", range(n), lowBound=0, upBound=n - 1, cat='Continuous')
    for i in range(1, n-1):
        for j in range(1, n-1):
            if i != j:
                prob += u[i] - u[j] + n * x[i, j] <= n - 1

    # Solve the ILP
    prob.solve()

    # Extract the path from the solution
    path = [0]
    current = 0
    while current != n-1:
        for j in range(n):
            if current != j and pulp.value(x[current, j]) == 1:
                path.append(j)
                current = j
                break

    # Calculate the total distance
    total_distance = sum(adj_matrix[path[i]][path[i+1]] for i in range(n - 1))

    return path, total_distance

### Utils

In [44]:
# Points to adj matrix
def points_to_adj_matrix(points):
    n = len(points)
    adj_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            adj_matrix[i][j] = adj_matrix[j][i] = math.dist(points[i], points[j])
    return adj_matrix

### Quick Test

In [45]:
# The adjacency matrix of the graph
points = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
adj_matrix = points_to_adj_matrix(points)

print(held_karp_solver(adj_matrix)) # Output: 21
print(bf_solver(adj_matrix)) # Output: 21


([0, 1, 2, 3, 4], 5.656854249492381)
([0, 1, 2, 3, 4], 5.656854249492381)


In [46]:
# Test with random points
import random
import time

points = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(10)]
adj_matrix = points_to_adj_matrix(points)

start_time = time.time()
print(held_karp_solver(adj_matrix))
print("Held-Karp time:", time.time() - start_time)

start_time = time.time()
print(bf_solver(adj_matrix))
print("BF time:", time.time() - start_time)

([0, 7, 2, 4, 5, 3, 6, 1, 8, 9], 270.0981488625184)
Held-Karp time: 0.007207393646240234
([0, 7, 2, 4, 5, 3, 6, 1, 8, 9], 270.0981488625184)
BF time: 0.07203197479248047


### Testing the implementation

In [47]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
from tqdm import tqdm

In [49]:
# Test for many cases, this is slow due to BF
for length in range(0, 13, 3):
    for _ in tqdm(range(int(10 / (length + 1)) + 5), desc=f"Testing length: {length}"):
        points = [(random.uniform(-100, 100), random.uniform(-100, 100)) for _ in range(length)]
        adj_matrix = points_to_adj_matrix(points)
        
        held_karp_res = held_karp_solver(adj_matrix)
        held_karp_set_res = held_karp_solver_using_set(adj_matrix)
        # tsp_ilp_solver_res = tsp_ilp_solver(adj_matrix)
        bf_res = bf_solver(adj_matrix)

        # Assert the results compare to BF solver
        assert held_karp_res[0] == bf_res[0], f"HK != BF for path, {held_karp_res[0]} != {bf_res[0]}"
        assert held_karp_set_res[0] == bf_res[0], f"HK Set != BF for path, {held_karp_set_res[0]} != {bf_res[0]}"
        # assert tsp_ilp_solver_res[0] == bf_res[0], f"ILP != BF for path, {tsp_ilp_solver_res[0]} != {bf_res[0]}"

        assert held_karp_res[1] == bf_res[1], f"HK != BF for distance, {held_karp_res[1]} != {bf_res[1]}"
        assert held_karp_set_res[1] == bf_res[1], f"HK Set != BF for distance, {held_karp_set_res[1]} != {bf_res[1]}"
        # assert tsp_ilp_solver_res[1] == bf_res[1], f"ILP != BF for distance, {tsp_ilp_solver_res[1]} != {bf_res[1]}"

print("Passed")

Testing length: 12: 100%|██████████| 5/5 [00:42<00:00,  8.43s/it]

Passed


In [50]:
# Testing multiple ways for implementing the Held-Karp algorithm
avg_time_cache = 0
avg_time_set = 0
# avg_ilp_time = 0

for _ in range(10):
    points = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(10)]
    adj_matrix = points_to_adj_matrix(points)

    start_time = time.time()
    held_karp_solver(adj_matrix)
    avg_time_cache += time.time() - start_time
    print("Held-Karp time:", time.time() - start_time)

    start_time = time.time()
    held_karp_solver_using_set(adj_matrix)
    print("Held-Karp Set time:", time.time() - start_time)
    avg_time_set += time.time() - start_time

    # start_time = time.time()
    # tsp_ilp_solver((adj_matrix[:-1])[:][:-1])
    # print("ILP time:", time.time() - start_time)
    # avg_ilp_time += time.time() - start_time

print("Average time for cache:", avg_time_cache / 10)
print("Average time for set:", avg_time_set / 10)
# print("Average time for ILP:", avg_ilp_time / 10)

Held-Karp time: 0.015782833099365234
Held-Karp Set time: 0.023154497146606445
Held-Karp time: 0.016842126846313477
Held-Karp Set time: 0.027333974838256836
Held-Karp time: 0.009853839874267578
Held-Karp Set time: 0.015386819839477539
Held-Karp time: 0.011191368103027344
Held-Karp Set time: 0.19412732124328613
Held-Karp time: 0.010795354843139648
Held-Karp Set time: 0.01575446128845215
Held-Karp time: 0.011000871658325195
Held-Karp Set time: 0.016983985900878906
Held-Karp time: 0.016232013702392578
Held-Karp Set time: 0.01798558235168457
Held-Karp time: 0.014143943786621094
Held-Karp Set time: 0.01803302764892578
Held-Karp time: 0.013238906860351562
Held-Karp Set time: 0.01814723014831543
Held-Karp time: 0.01293325424194336
Held-Karp Set time: 0.021781206130981445
Average time for cache: 0.013201451301574707
Average time for set: 0.03697414398193359
